<a href="https://colab.research.google.com/github/carascott/skills-introduction-to-github/blob/main/DataProject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Fetching/downloading data as a csv file
import pandas as pd
import requests
import json
import sqlite3

# https://opendata.charlottesville.org/datasets/304f8e27f2ca44aa989d58846db56b38_24/explore

df = pd.read_csv('/content/WaterUsageCville.csv')
#print(df.head())

In [20]:
# Allowing user to convert from csv to either JSON or SQL

file_type = input("Enter desired file type (sql or json): ")

if file_type.lower() == "json":
  # Convert to JSON
  json_data = df.to_json(orient='records')
  print("Data converted to JSON")

elif file_type.lower() == "sql":
  # Convert to SQL
  conn = sqlite3.connect('water_usage.db')
  df.to_sql('water_usage_table', conn, if_exists='replace', index=False)
  conn.close()
  print("Data converted to SQLite database (water_usage.db)")

else:
  print("Invalid file type specified.")

Enter desired file type (sql or json): json
Data converted to JSON


In [12]:
# Converting csv file to json

json_data = df.to_json(orient='records')
print(json_data)
print("CSV data successfully converted to JSON")

[{"UtilityConsumptionID":1,"Neighborhood":"North Downtown","June2018":1483572,"July2018":1383526,"August2018":1447082,"September2018":1674497,"October2018":1206079,"November2018":1406674,"December2018":1278349,"January2019":1127121,"February2019":1152809,"March2019":1042641,"April2019":1260530},{"UtilityConsumptionID":2,"Neighborhood":"Martha Jefferson","June2018":515021,"July2018":528651,"August2018":518084,"September2018":575989,"October2018":458728,"November2018":538587,"December2018":505504,"January2019":438123,"February2019":482170,"March2019":438866,"April2019":474798},{"UtilityConsumptionID":3,"Neighborhood":"Woolen Mills","June2018":253210,"July2018":274249,"August2018":272838,"September2018":280603,"October2018":218916,"November2018":240953,"December2018":237290,"January2019":221760,"February2019":206131,"March2019":191738,"April2019":208625},{"UtilityConsumptionID":4,"Neighborhood":"Locust Grove","June2018":516853,"July2018":489506,"August2018":520905,"September2018":541363,"

In [21]:
# Printing JSON file
with open('WaterUsageCville.json', 'r') as f:
    data = json.load(f)

for i in range(min(5, len(data))):
    print(data[i])

{'UtilityConsumptionID': 1, 'Neighborhood': 'North Downtown', 'June2018': 1483572, 'July2018': 1383526, 'August2018': 1447082, 'September2018': 1674497, 'October2018': 1206079, 'November2018': 1406674, 'December2018': 1278349, 'January2019': 1127121, 'February2019': 1152809, 'March2019': 1042641, 'April2019': 1260530}
{'UtilityConsumptionID': 2, 'Neighborhood': 'Martha Jefferson', 'June2018': 515021, 'July2018': 528651, 'August2018': 518084, 'September2018': 575989, 'October2018': 458728, 'November2018': 538587, 'December2018': 505504, 'January2019': 438123, 'February2019': 482170, 'March2019': 438866, 'April2019': 474798}
{'UtilityConsumptionID': 3, 'Neighborhood': 'Woolen Mills', 'June2018': 253210, 'July2018': 274249, 'August2018': 272838, 'September2018': 280603, 'October2018': 218916, 'November2018': 240953, 'December2018': 237290, 'January2019': 221760, 'February2019': 206131, 'March2019': 191738, 'April2019': 208625}
{'UtilityConsumptionID': 4, 'Neighborhood': 'Locust Grove', 'J

In [16]:
# Converting csv to sql database

conn = sqlite3.connect('water_usage.db')
cursor = conn.cursor()

columns = df.columns.tolist()

create_table_query = f"""
CREATE TABLE IF NOT EXISTS water_usage_table (
    {', '.join([f'{col} TEXT' for col in columns])}
)
"""
cursor.execute(create_table_query)

for index, row in df.iterrows():
    placeholders = ', '.join(['?' for _ in columns])
    insert_query = f"INSERT INTO water_usage_table VALUES ({placeholders})"
    cursor.execute(insert_query, tuple(row))

conn.commit()
conn.close()

print("CSV data successfully imported into the SQLite database.")

CSV data successfully imported into the SQLite database.


In [26]:
# Allow user to input file name and file type, then convert to either json, csv, or sql

def convert_file(input_file, input_type, output_type):
    try:
        if input_type.lower() == "csv":
            df = pd.read_csv(input_file)
        elif input_type.lower() == "json":
            df = pd.read_json(input_file)
        elif input_type.lower() == "sql":
            conn = sqlite3.connect(input_file)
            df = pd.read_sql_query("SELECT * FROM water_usage_table", conn)
            conn.close()
        else:
            print("Invalid input file type.")
            return

        if output_type.lower() == "json":
            output_file = input_file.replace(input_file.split(".")[-1], "json") # change the extension of the input file to json
            df.to_json(output_file, orient='records')
            print(f"Data converted to JSON and saved to {output_file}")
        elif output_type.lower() == "sql":
            output_file = input_file.replace(input_file.split(".")[-1], "db")  # change the extension of the input file to db
            conn = sqlite3.connect(output_file)
            df.to_sql('water_usage_table', conn, if_exists='replace', index=False)
            conn.close()
            print(f"Data converted to SQLite database and saved to {output_file}")
        elif output_type.lower() == "csv":
            output_file = input_file.replace(input_file.split(".")[-1], "csv") # change the extension of the input file to csv
            df.to_csv(output_file, index=False)
            print(f"Data converted to CSV and saved to {output_file}")
        else:
            print("Invalid output file type.")

    except FileNotFoundError:
        print(f"Error: Input file '{input_file}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    input_file = input("Enter the input file name: ")
    input_type = input("Enter the input file type (csv, json, or sql): ")
    output_type = input("Enter the desired output file type (csv, json, or sql): ")

    convert_file(input_file, input_type, output_type)

Enter the input file name: WaterUsageCville.csv
Enter the input file type (csv, json, or sql): csv
Enter the desired output file type (csv, json, or sql): json
Data converted to JSON and saved to WaterUsageCville.json


In [37]:
# Asking users which file to use, if they want to add or delete a column
# (if add: input the new name & add new data, if delete: input the name of the column you want deleted),
# ask if they want the new file to be saved, and if yes, write it to sql database regardless of file type used,
# finally output the # of records and # of columns of both the original file inputted and the final sql saved

def modify_dataframe(file_path):
    try:
        if file_path.lower().endswith('.csv'):
            df_original = pd.read_csv(file_path)
        elif file_path.lower().endswith(('.json')):
            df_original = pd.read_json(file_path)
        elif file_path.lower().endswith(('.db')):
            conn = sqlite3.connect(file_path)
            df_original = pd.read_sql_query("SELECT * FROM water_usage_table", conn)
            conn.close()
        else:
            print("Unsupported file type. Please provide a CSV, JSON, or SQLite database file.")
            return

        df = df_original.copy() #create a copy to work with

        action = input("Do you want to add or delete a column? (add/delete): ").lower()
        if action == "add":
            column_name = input("Enter the name of the new column: ")
            default_value = input("Enter the default value for the new column: ")
            df[column_name] = default_value
            print(f"Column '{column_name}' added with default value '{default_value}'")

        elif action == "delete":
            print("Available columns:")
            for col in df.columns:
                print(col)
            column_to_delete = input("Enter the name of the column to delete: ")
            if column_to_delete in df.columns:
                df = df.drop(column_to_delete, axis=1)
                print(f"Column '{column_to_delete}' deleted successfully.")
            else:
                print(f"Error: Column '{column_to_delete}' not found in the DataFrame.")

        else:
            print("Invalid action specified. Please enter 'add' or 'delete'.")

        print("\nModified DataFrame:")
        print(df.head())

        save_changes = input("Do you want to save the changes to the SQL database? (yes/no): ").lower()
        if save_changes == "yes":
            conn = sqlite3.connect('water_usage_modified.db')
            df.to_sql('water_usage_table', conn, if_exists='replace', index=False)
            conn.close()
            print("Changes saved to 'water_usage_modified.db'")

        # Outputting the data
        print("\nOriginal file:")
        print("Number of records:", len(df_original))
        print("Number of columns:", len(df_original.columns))

        conn = sqlite3.connect('water_usage_modified.db')
        df_sql = pd.read_sql_query("SELECT * FROM water_usage_table", conn)
        conn.close()

        print("\nFinal SQL database:")
        print("Number of records:", len(df_sql))
        print("Number of columns:", len(df_sql.columns))


    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    file_name = input("Enter the file name (including extension): ")
    modify_dataframe(file_name)

Enter the file name (including extension): WaterUsageCville.csv
Do you want to add or delete a column? (add/delete): delete
Available columns:
UtilityConsumptionID
June2018
July2018
August2018
September2018
October2018
November2018
December2018
January2019
February2019
March2019
April2019
Enter the name of the column to delete: April2019
Column 'April2019' deleted successfully.

Modified DataFrame:
   UtilityConsumptionID  June2018  July2018  August2018  September2018  \
0                     1   1483572   1383526     1447082        1674497   
1                     2    515021    528651      518084         575989   
2                     3    253210    274249      272838         280603   
3                     4    516853    489506      520905         541363   
4                     5    461108    545330      437904         487358   

   October2018  November2018  December2018  January2019  February2019  \
0      1206079       1406674       1278349      1127121       1152809   
1      